# 🚀 Day 2 실습 1: Naive RAG 베이스라인 - 파스텔 문제 발견

## 🎯 학습 목표
- **단순 질문 vs 복합 질문**으로 Naive RAG의 한계 직접 체험
- **벡터 검색**만으로는 부족한 이유 이해
- **02-06번 개선 여정**의 출발점 설정

### 💡 핵심 시나리오
사용자: **"내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"**

이 복합 질문이 왜 어려울까요? 지금부터 확인해봅시다!

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas numpy

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import numpy as np

print("✅ 라이브러리 준비 완료!")

## 📚 파스텔 문서 준비

### 동음이의어 "파스텔"의 세 얼굴
사용자가 "파스텔"이라고 하면... 무엇을 말하는 걸까요?

In [ ]:
# 파스텔 문서 + 노이즈 문서 (8개)
pastel_docs = [
    {
        "content": """파스텔 호텔 예약 안내
서울 강남구에 위치한 파스텔 호텔입니다.
- 예약 변경: 투숙 3일 전까지 가능 (위약금 없음)
- 주말 요금: 평일 대비 30% 할인""",
        "type": "hotel"
    },
    {
        "content": """파스텔 카페 메뉴
홍대 파스텔 카페는 디저트로 유명합니다.
- 예약: 단체 예약만 가능 (10명 이상)
- 주말: 마카롱 만들기 체험 (14:00-16:00)""",
        "type": "cafe"
    },
    {
        "content": """파스텔 소극장 공연 안내
대학로 파스텔 소극장에서 뮤지컬을 상영합니다.
- 티켓 예약: 인터파크, 현장 구매 가능
- 주말: 토요일 15:00 추가 공연""",
        "type": "theater"
    },
    {
        "content": """국립현대미술관 전시 정보
서울 국립현대미술관 덕수궁관 전시 안내입니다.
- 주말 관람 팁: 토요일 14:00-16:00 혼잡, 일요일 오전 10:00-12:00 한적함""",
        "type": "museum"
    },
    {
        "content": """서울 여행 블로그
미술관은 주말 오전이 베스트 타이밍.
예약은 미리미리! 특히 숙소는 필수.""",
        "type": "blog"
    },
    {
        "content": """서울역 교통 안내
KTX, 지하철 1/4호선 환승 가능.
주말 막차 시간: 자정까지 운행""",
        "type": "transport"
    },
    {
        "content": """주말 날씨 예보
이번 주말 서울 날씨는 맑음.
외출하기 좋은 날씨입니다.""",
        "type": "weather"
    },
    {
        "content": """예약 플랫폼 가이드
네이버, 카카오 예약 시스템 비교.
주말 예약은 평일보다 2배 빠르게 마감됩니다.""",
        "type": "guide"
    }
]

print("📚 문서 8개 준비 완료 (파스텔 5개 + 노이즈 3개)")
for i, doc in enumerate(pastel_docs, 1):
    print(f"  {i}. [{doc['type']}] {doc['content'].split(chr(10))[0]}")

## 🔍 Naive RAG 시스템 구축

가장 단순한 방법: **벡터 검색만** 사용

In [ ]:
# Document 객체로 변환
documents = [Document(page_content=doc["content"], metadata={"type": doc["type"]}) 
             for doc in pastel_docs]

# 임베딩 모델 + 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ Naive RAG 시스템 구축 완료!")
print(f"   임베딩 모델: BAAI/bge-m3")
print(f"   문서 수: {len(documents)}개")

## 🧪 테스트 1: 단순 질문 (성공 케이스)

먼저 간단한 질문으로 RAG가 잘 동작하는지 확인해봅시다:

In [ ]:
# 단순 질문 2개 테스트
simple_queries = [
    "파스텔 호텔 예약 변경 방법은?",
    "주말에 미술관 언제 가면 좋아?"
]

for query in simple_queries:
    print(f"❓ 질문: {query}")
    results = vectorstore.similarity_search(query, k=1)
    
    doc = results[0]
    doc_type = doc.metadata.get("type", "unknown")
    print(f"✅ 검색 성공: [{doc_type.upper()}] {doc.page_content.split(chr(10))[0]}")
    print()

## 🧪 테스트 2: 복합 질문 (실패 케이스)

이제 **3가지 정보를 동시에** 요구하는 질문을 던져봅시다:

In [ ]:
# 복합 질문 (3가지 정보 필요)
complex_query = "내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"

print("❓ 복합 질문:")
print(f"   {complex_query}\n")

print("📋 필요한 정보 (3개):")
needed = ["theater", "hotel", "cafe"]
for need in needed:
    print(f"   - {need}")

print("\n🔍 Naive RAG 검색 결과 (Top-3):\n")

# 벡터 검색
results = vectorstore.similarity_search(complex_query, k=3)

# 상세 출력
retrieved_types = []
for i, doc in enumerate(results, 1):
    doc_type = doc.metadata.get("type", "unknown")
    retrieved_types.append(doc_type)
    first_line = doc.page_content.split("\n")[0]
    print(f"{i}. [{doc_type.upper()}] {first_line}")
    print(f"   {doc.page_content[:80]}...")
    print()

# 성공/실패 체크
print("━" * 50)
print("📊 검색 결과:")
print(f"   필요: {needed}")
print(f"   검색됨: {retrieved_types}")
print()

print("💡 개별 체크:")
for need in needed:
    if need in retrieved_types:
        print(f"   ✅ {need} - 찾음")
    else:
        print(f"   ❌ {need} - 못 찾음")

found_count = sum(1 for need in needed if need in retrieved_types)
print(f"\n🎯 결과: {found_count}/3 성공 → ", end="")
if found_count == 3:
    print("✅ 완벽!")
else:
    print("❌ 실패!")

## 🚀 다음 단계: Advanced RAG로 개선하기

### 이제부터 단계별로 해결합니다!

| 노트북 | 해결하는 문제 | 핵심 기법 |
|--------|---------------|----------|
| **02** | 실패 원인 분석 | 파스텔 케이스 심층 분석 |
| **03** | 멀티홉 추론 | Multi-Query Retrieval |
| **04** | 동음이의어 | Metadata Filtering |
| **05** | 검색 정확도 | Hybrid Search + Re-ranking |
| **06** | 지능형 시스템 | Modular RAG + Routing |

---

### 💡 핵심 인사이트

**Naive RAG는 "검색만" 합니다.**
- 질문 분해? ❌
- 맥락 이해? ❌  
- 정보 연결? ❌
- 노이즈 제거? ❌

**Advanced RAG는 "생각합니다".**
- 질문 분해 ✅
- 맥락 이해 ✅
- 정보 연결 ✅
- 노이즈 제거 ✅

**🎯 이제 02번 노트북에서 실패 원인을 더 깊이 파헤쳐봅시다!**

In [ ]:
# 전체 성능 비교 (성공 vs 실패)
test_cases = [
    {"query": "파스텔 호텔 예약 변경", "needed": ["hotel"], "category": "단순"},
    {"query": "주말 미술관 시간", "needed": ["museum"], "category": "단순"},
    {"query": complex_query, "needed": ["theater", "hotel", "cafe"], "category": "복합"}
]

results_table = []

for case in test_cases:
    query = case["query"]
    needed = case["needed"]
    category = case["category"]
    
    # 검색
    docs = vectorstore.similarity_search(query, k=3)
    found_types = [doc.metadata.get("type") for doc in docs]
    
    # 완성도 체크
    found_all = all(need in found_types for need in needed)
    
    results_table.append({
        "유형": category,
        "질문": query[:30] + "..." if len(query) > 30 else query,
        "필요": ",".join(needed),
        "검색됨": ",".join(found_types),
        "완성도": "✅" if found_all else "❌"
    })

# 표로 출력
df = pd.DataFrame(results_table)
print("📊 Naive RAG 성능 비교:\n")
print(df.to_string(index=False))

# 유형별 성공률
simple_success = sum(1 for r in results_table[:2] if r["완성도"] == "✅") / 2
complex_success = sum(1 for r in results_table[2:] if r["완성도"] == "✅") / 1

print(f"\n📈 성공률:")
print(f"   단순 질문: {simple_success:.0%} ✅")
print(f"   복합 질문: {complex_success:.0%} ❌")

## 🔍 왜 실패했을까?

위 테스트에서 **theater, hotel, cafe 3개 중 일부만** 검색되거나 **아예 다른 문서**가 검색되었습니다.

### Naive RAG의 3가지 근본적 한계

#### 1️⃣ 동음이의어 문제
- **"파스텔"** → 호텔? 카페? 소극장? 
- 벡터 검색은 **맥락 이해 부족**
- "공연", "호텔", "카페" 각각을 독립적으로만 처리

#### 2️⃣ 멀티홉 추론 실패  
- 질문 = **3개 요구사항** (공연 + 호텔 + 카페)
- Naive RAG = **단일 검색**만 수행
- Top-3로는 **3개 정보를 모두 못 찾음**

#### 3️⃣ 노이즈 문서 간섭
- 관련 없는 문서(교통, 날씨, 가이드)가 **상위권에 끼어듦**
- "주말", "예약" 같은 **일반 키워드에 반응**
- 진짜 필요한 문서가 **밀려남**

---

### 📊 성능 요약

## 🚀 다음 단계: Advanced RAG로 개선하기

### 이제부터 단계별로 해결합니다!

| 노트북 | 해결하는 문제 | 핵심 기법 |
|--------|---------------|----------|
| **02** | 실패 원인 분석 | 파스텔 케이스 심층 분석 |
| **03** | 멀티홉 추론 | Multi-Query Retrieval |
| **04** | 동음이의어 | Metadata Filtering |
| **05** | 검색 정확도 | Hybrid Search + Re-ranking |
| **06** | 지능형 시스템 | Modular RAG + Routing |

---

### 💡 핵심 인사이트

**Naive RAG는 "검색만" 합니다.**
- 질문 분해? ❌
- 맥락 이해? ❌  
- 정보 연결? ❌

**Advanced RAG는 "생각합니다".**
- 질문 분해 ✅
- 맥락 이해 ✅
- 정보 연결 ✅

**🎯 이제 02번 노트북에서 실패 원인을 더 깊이 파헤쳐봅시다!**